In [ ]:
pip install plotly.express ipywidgets dash

In [49]:
#@title Run this
from google.colab import output
output.no_vertical_scroll()

<IPython.core.display.Javascript object>

In [32]:
#@title Plot Functions (just run them)
import plotly.graph_objects as go
import numpy as np

def PlotUniswapV2Sample(fig):
    AAmount = np.linspace(A//3,2*A, 100)
    BAmount = LSquare / AAmount
    fig.add_trace(go.Scatter(x=AAmount, y=BAmount))


def PlotDotWithName(fig, P, text):
    fig.add_trace(go.Scatter(x=[P[0]], y=[P[1]],
                  text=[text],
                  textposition="top right",
                  mode='markers+text'))

def PlotLinesToAxis(fig, P):
    lineFormat = dict(color='firebrick', width=2, dash='dot')
    fig.add_trace(go.Scatter(x=[P[0],0], y=[P[1], P[1]], line = lineFormat))
    fig.add_trace(go.Scatter(x=[P[0],P[0]], y=[P[1], 0], line = lineFormat))


def FormatFig(fig, title, width = 500, height = 500):
   fig.update_layout(title=title,
                     width=width,
                     height=height,
                     showlegend = False,
                     xaxis_title="ETH Amount", yaxis_title="USDC Amount",
)

For the purpose of this chapter asumme that we have
* `A` the balance of the token `X`
* `B` the balance of the token `Y`
Assign those values to the following cell to test by yourself.

Your pool has to always keep the constant equality
$AB = L^2 (1)$

Suppose a trader wants to buy an amount $a$ of token $X$.
Therefore, he will have to pay $b$ in $Y$, so, to calculate this, remember the pool will always keep equality (1), so

$$(A-a)(B-b) = L^2 (2)$$
$$AB - aB -Ab + ab = L^2$$
But by (1),
$$ab - aB -Ab = 0$$
So if we isolate $b$,
$$b= \frac{aB}{A-a}$$
In the same way, if you want to buy an amount $b$ of token $Y$, you can isolate $a$ in (2) the same way was done for $b$ and obtain
$$a= \frac{bA}{B-b}$$

In [26]:
#@title Add your variables
import ipywidgets as widgets
from IPython.display import display
A,B,a = 100,4000000,20
X,Y = "ETH","USDC"
text_input_A = widgets.Text(description='A:',placeholder='Add the amount of X tokens you want in the pool')
button_A = widgets.Button(description="Update A")

text_input_B = widgets.Text(description='B:',placeholder='Add the amount of Y tokens you want in the pool')
button_B = widgets.Button(description="Update B")

text_input_a = widgets.Text(description='a:',placeholder='Add the amount of a to buy of X')
button_a = widgets.Button(description="Update a")

text_input_X = widgets.Text(description='X:',placeholder='Name of X')
button_X = widgets.Button(description="Rename X")

text_input_Y = widgets.Text(description='Y:',placeholder='Name of Y')
button_Y = widgets.Button(description="Rename Y")

def setA(val):
    global A
    A = int(text_input_A.value)
def setB(val):
    global B
    B = int(text_input_B.value)
def seta(val):
    global a
    a = int(text_input_a.value)
def setX(val):
    global X
    X = text_input_X.value
def setY(val):
    global Y
    Y = text_input_Y.value

button_A.on_click(setA)
button_B.on_click(setB)
button_a.on_click(seta)
button_X.on_click(setX)
button_Y.on_click(setY)

row_A = widgets.HBox([text_input_A, button_A])
row_B = widgets.HBox([text_input_B, button_B])
row_a = widgets.HBox([text_input_a, button_a])
row_X = widgets.HBox([text_input_X, button_X])
row_Y = widgets.HBox([text_input_Y, button_Y])

display(row_A)
display(row_B)
display(row_a)
display(row_X)
display(row_Y)

In [27]:
#@title Uniswap V2 backlog
def poolSwapNoFee(a):
    b = a*B / (A - a)
    return b

print(f"""A = {A}
B = {B}
AB = {A*B}
L^2 = {A*B}

You want an amount of a = {a} in {X}.
You will need to pay {poolSwapNoFee(a)} in {Y}
""")

LSquare = A*B
b = poolSwapNoFee(a)

A = 100
B = 4000000
AB = 400000000
L^2 = 400000000

You want an amount of a = 20 in ETH.
You will need to pay 1000000.0 in USDC



In [28]:
#@title Visialization of the example given
P = [A,B]
APrime = A - a
BPrime = B + b
Q = [APrime, BPrime]


fig = go.Figure()
PlotUniswapV2Sample(fig)
PlotDotWithName(fig,P,"P")
PlotDotWithName(fig,Q,"Q")
PlotLinesToAxis(fig, P)
PlotLinesToAxis(fig, Q)
FormatFig(fig, "Before and After a Tx")
fig.show()

Now, let
* $p$ the spot prince of token $X$ in terms of $Y$ and
* $p_e(b)$ the effective price paid by the trader when depositing an amount $b$ of $Y$.
They can be calculated with the equations
$$p_e(b) = \frac{b}{a} = \frac{B}{A} + \frac{b}{A}$$
$$p = \frac{B}{A} $$
The **spot price** can be understood as the price the liquidity pool offers traders at a particular moment.

| The difference between the amount $b$ that a trader pays and the amount that the trader would have paid if they had bought an amount $a$ at a price equal to the spot price $p$ is called **price impact**
$$Price\ Impact = b- ap = a\left(\frac{b}{a} - p\right) = a\left(p_e(b) - p\right)  = \frac{ab}{A} $$

In [57]:
#@title Interaction with all parameters
from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px

app = Dash(__name__)

minA = A//2
maxA = A

app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.Slider(
        A//3,
        A//1.5,
        step=None,
        value=a,
        id='aamount-slider'
    )
])


@callback(
    Output('graph-with-slider', 'figure'),
    Input('aamount-slider', 'value'))
def update_figure(amount_wanted):
    APrime = A - amount_wanted
    BPrime = B + poolSwapNoFee(amount_wanted)

    PointB = [0,B]
    PointA = [A,0]
    PointBPlusb = [0,BPrime]
    PointAMinusa = [APrime, 0]
    PointPPrime = [APrime, BPrime]

    fig = go.Figure()
    PlotUniswapV2Sample(fig)

    PlotDotWithName(fig,P,"P")
    PlotDotWithName(fig,PointPPrime,"P'")

    PlotDotWithName(fig,PointA,"A")
    PlotDotWithName(fig,PointAMinusa,"A-a")
    PlotDotWithName(fig,PointB,"B")
    PlotDotWithName(fig,PointBPlusb,"B+b")


    PlotLinesToAxis(fig, P)
    PlotLinesToAxis(fig, Q)
    FormatFig(fig, "Interaction with all parameters", width = 700, height = 700)
    return fig


if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>